In [2]:
!pip install anthropic



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import os
from anthropic import Anthropic


anthropic_client = Anthropic(
    api_key=os.environ.get("ANTHROPIC_API_KEY")  
)


CLAUDE_MODEL = "claude-3-5-sonnet-20240620"  
MAX_TOKENS = 4096  

print("API postavke uspešno učitane!")

API postavke uspešno učitane!


In [4]:


def extract_elements(text, tag_name):
    """
    Izvlači sadržaj iz XML taga.
    """
    import re
    
    pattern = f"<{tag_name}>(.*?)</{tag_name}>"
    matches = re.findall(pattern, text, re.DOTALL)
    
    return matches


STANDARD_CRITERIA = """
Below are the standard evaluation criteria you must assess in every evaluation:
- Relevance (0–10): Response directly addresses the user's query and is on-topic.
- Accuracy (0–10): Information provided is factually correct and up-to-date.
- Coherence (0–10): Response is logically structured and easy to follow.
- Conciseness (0–10): Answer is appropriately brief without unnecessary elaboration.
- Clarity (0–10): Content is presented in a clear, easily understandable way.
- Completeness (0–10): Response addresses all aspects of the user's query.
- Language Appropriateness (0–10): Language level and tone match the target audience.
- Chain-of-Thought Reasoning (0–10): Response demonstrates logical step-by-step reasoning using expressions like "first", "then", "because", etc.
"""


def generate_criteria(agent_system_prompt):
    """
    Generiše dinamičke kriterijume evaluacije na osnovu system prompta AI agenta.
    Kombinuje ih sa standardnim kriterijumima.
    """
    raw_prompt = f"""
You are an expert AI analyst specializing in the evaluation and interpretation of system prompts for AI agents. Your primary task is to analyze a given system prompt and extract ONLY specific, unique evaluation criteria that an AI agent should follow when responding to user input.

Here is the system prompt you need to analyze:

<system_prompt>
{agent_system_prompt}
</system_prompt>

{STANDARD_CRITERIA}

Your task is to identify ADDITIONAL DYNAMIC criteria that are specific to this particular system prompt and are not covered by the standard criteria above.

Your analysis should follow these steps:

1. Carefully read and understand the system prompt.

2. In <prompt_breakdown> tags, perform a detailed analysis of the prompt:
   a. Summarize the main goals of the system prompt  
   b. List key phrases or instructions that imply criteria  
   c. Identify any specific rules or constraints mentioned  
   d. Note any ethical considerations or guidelines  

3. In <criteria_brainstorm> tags, list potential DYNAMIC criteria based on your breakdown. Focus ONLY on criteria that are specific to this prompt and NOT covered by the standard criteria. Number each criterion.

4. For each criterion, decide on the appropriate scoring scale:
   - Use a binary scale (0-1) for criteria that can only be assessed as present/absent or met/not met
   - Use a continuous scale (0-10) for criteria that can have varying degrees of quality or implementation

5. Format your final output as follows:

<dynamic_criteria>
- [Criterion Name 1] (Dynamic): [Brief explanation] [Scale: 0-1 or 0-10]
- [Criterion Name 2] (Dynamic): [Brief explanation] [Scale: 0-1 or 0-10]
...
</dynamic_criteria>

Important guidelines:
- Generate ONLY dynamic criteria specific to this prompt (typically 3-8 criteria)
- Assign an appropriate scale to each criterion based on its nature:
  * Binary scale (0-1) for yes/no criteria (compliance with specific rules)
  * Continuous scale (0-10) for quality-based criteria (how well something is done)
- Be thoughtful and precise in your analysis
- Do not invent irrelevant criteria. Use the system prompt as your sole source.
- Ensure that each criterion is clearly derived from the content of the system prompt.
- DO NOT include standard criteria or variations of them

Begin your response with the <prompt_breakdown> section, followed by the <criteria_brainstorm> section, and conclude with the <dynamic_criteria> output.
"""
    
    response = anthropic_client.messages.create(
        model=CLAUDE_MODEL,
        max_tokens=MAX_TOKENS,
        messages=[
            {"role": "user", "content": raw_prompt}
        ],
        temperature=0.3  
    )
    
   
    content = response.content[0].text
    dynamic_criteria = extract_elements(content, "dynamic_criteria")
    
  
    combined_criteria = STANDARD_CRITERIA
    
    if dynamic_criteria and dynamic_criteria[0].strip():
        combined_criteria += "\n\nAdditional dynamic criteria specific to this system prompt:\n" + dynamic_criteria[0]
    
    return combined_criteria


if __name__ == "__main__":
    test_prompt = """
    Ti si stručni nutricionista koji daje savete o zdravoj ishrani.
    1. Uvek navedi naučno dokazane informacije
    2. Izbegavaj promotivne savete za komercijalne dijete
    3. Prilagodi savete individualnim potrebama korisnika
    4. Koristi jednostavan i razumljiv jezik
    5. Ako je pitanje van domena ishrane, ljubazno usmeri korisnika na odgovarajućeg stručnjaka
    """
    
    combined_criteria = generate_criteria(test_prompt)
    
    print("=== KOMBINOVANI KRITERIJUMI ===")
    print(combined_criteria)

print("Generator kriterijuma sa fleksibilnom skalom za dinamičke kriterijume uspešno postavljen!")

=== KOMBINOVANI KRITERIJUMI ===

Below are the standard evaluation criteria you must assess in every evaluation:
- Relevance (0–10): Response directly addresses the user's query and is on-topic.
- Accuracy (0–10): Information provided is factually correct and up-to-date.
- Coherence (0–10): Response is logically structured and easy to follow.
- Conciseness (0–10): Answer is appropriately brief without unnecessary elaboration.
- Clarity (0–10): Content is presented in a clear, easily understandable way.
- Completeness (0–10): Response addresses all aspects of the user's query.
- Language Appropriateness (0–10): Language level and tone match the target audience.
- Chain-of-Thought Reasoning (0–10): Response demonstrates logical step-by-step reasoning using expressions like "first", "then", "because", etc.


Additional dynamic criteria specific to this system prompt:

- Scientific Evidence Base (Dynamic): Extent to which the response cites or refers to scientific evidence [Scale: 0-10]
- 

In [5]:
def generate_questions(agent_system_prompt):
    """
    Generiše set od 20 test pitanja na osnovu system prompta AI agenta.
    """
    raw_prompt = f"""
You are an expert AI evaluation specialist tasked with creating a comprehensive set of test questions to assess an AI chatbot's capabilities. Your goal is to generate 20 diverse and challenging questions that will thoroughly evaluate the chatbot's performance based on its intended function.

<system_prompt>
{agent_system_prompt}
</system_prompt>

Your task is to analyze this system prompt and create questions that will effectively test the chatbot's abilities within its defined role and purpose.

Before generating the final list of questions, wrap your evaluation planning process inside <evaluation_planning> tags. Consider the following:

1. Analyze the system prompt:
   - Identify the chatbot's primary function and domain of expertise  
   - List key capabilities mentioned in the prompt  
   - Note any specific constraints or guidelines  

2. Categorize potential question types:
   - General knowledge or domain-specific expertise  
   - Logical reasoning and problem-solving  
   - Creativity and idea generation  
   - Conversational ability and interaction quality  
   - Accuracy and precision  
   - Consistency and clarity in responses  

3. Plan for question distribution:
   - Ensure a balance of different question types  
   - Include a range of difficulties (easy, medium, hard)  
   - Consider various formats (open-ended, multiple-choice, scenario-based)  

4. Specific areas of performance to evaluate:
   - List key performance indicators based on the chatbot's intended function  
   - Plan how to test these indicators through questions  

After your evaluation planning, generate a list of 20 questions that meet the following criteria:

1. Directly relevant to the chatbot's stated purpose  
2. Cover various aspects of the chatbot's performance, as identified in your planning  
3. Vary in complexity, including both straightforward and challenging questions  
4. Include a mix of open-ended and closed questions  
5. Be well-structured and clear, avoiding ambiguity  

Present your final list of questions in the <final_questions> tag, formatted as a numbered list.

Begin your response with the evaluation planning process:

<evaluation_planning>
[Your thought process here]
</evaluation_planning>

Then provide the final list of questions:

<final_questions>
1. [Question 1]
2. [Question 2]
...
20. [Question 20]
</final_questions>
"""
    
    response = anthropic_client.messages.create(
        model=CLAUDE_MODEL,
        max_tokens=MAX_TOKENS,
        messages=[
            {"role": "user", "content": raw_prompt}
        ],
        temperature=0.7  
    )
    
    return response.content[0].text


if __name__ == "__main__":
    test_prompt = """
    Ti si stručni nutricionista koji daje savete o zdravoj ishrani.
    1. Uvek navedi naučno dokazane informacije
    2. Izbegavaj promotivne savete za komercijalne dijete
    3. Prilagodi savete individualnim potrebama korisnika
    4. Koristi jednostavan i razumljiv jezik
    5. Ako je pitanje van domena ishrane, ljubazno usmeri korisnika na odgovarajućeg stručnjaka
    """
    
    questions_output = generate_questions(test_prompt)
    questions = extract_elements(questions_output, "final_questions")
    
    print("=== TEST PITANJA ===")
    if questions:
        print(questions[0])
    else:
        print("Nisu pronađena test pitanja")

print("Generator pitanja uspešno postavljen!")

=== TEST PITANJA ===

1. What are the main functions of protein in the human body?
2. Can you suggest a balanced meal plan for a vegetarian trying to increase their iron intake?
3. How does the glycemic index affect blood sugar levels, and which foods have a low glycemic index?
4. What are the potential health risks of following a ketogenic diet?
5. Could you explain the difference between soluble and insoluble fiber, and provide examples of foods rich in each?
6. How would you advise someone looking to lose weight in a healthy, sustainable manner?
7. What nutritional considerations should be taken into account for a pregnant woman in her first trimester?
8. Can you describe the role of omega-3 fatty acids in the body and list some good dietary sources?
9. How would you explain the concept of calorie density to someone trying to manage their weight?
10. What are some common nutritional deficiencies in a vegan diet, and how can they be addressed?
11. Could you provide a simple explanati

In [6]:

def evaluate_response(agent_description, agent_system_prompt, original_task, agent_response, combined_criteria):
    """
    Evaluira odgovor AI agenta na osnovu standardnih i dinamičkih kriterijuma.
    """
    
    eval_prompt = f"""
You are an expert AI evaluator tasked with providing a detailed and structured assessment of another AI agent's performance. Your evaluation must be objective, professional, and based strictly on the content of the agent's response in relation to the given task.

Here is the information you need to evaluate:

<agent_description>
{agent_description}
</agent_description>

<system_prompt>
{agent_system_prompt}
</system_prompt>

<original_task>
{original_task}
</original_task>

<ai_agent_response>
{agent_response}
</ai_agent_response>

<evaluation_criteria>
{combined_criteria}
</evaluation_criteria>

Your task is to evaluate the AI agent's response using all the criteria provided above. Conduct your analysis within <response_analysis> tags.

<response_analysis>
For each criterion:
- Quote relevant parts of the AI agent's response.
- Analyze how well the response meets the criterion.
- Justify your score based on the evidence from the response.
</response_analysis>

After your response analysis, provide your formal evaluation using the following XML structure:

<evaluation>
<criterion name="[Criterion Name]">
<justification>[Your justification]</justification>
<score>[Score]</score>
</criterion>
[Repeat for each criterion]
<overall_score>
<justification>[Summary of overall performance]</justification>
<score>[Average score]</score>
</overall_score>
</evaluation>

For Chain-of-Thought Reasoning criterion, specifically assess:
- Whether the response shows logical, step-by-step reasoning
- If the answer uses expressions like "first", "then", "because", "therefore", etc.
- How clearly and effectively the reasoning is structured
- Whether the reasoning improves understanding of the topic

Maintain a professional tone throughout your evaluation. Support each assessment with clear reasoning and concrete evidence from the AI agent's response. Avoid vague or general comments.

Begin your evaluation by analyzing the response in detail, then proceed to the formal evaluation structure.
"""
    
    response = anthropic_client.messages.create(
        model=CLAUDE_MODEL,
        max_tokens=MAX_TOKENS,
        messages=[
            {"role": "user", "content": eval_prompt}
        ],
        temperature=0.2  
    )
    
    return response.content[0].text



def extract_criteria_scores(evaluation_text):
    """
    Izvlači ocene za svaki kriterijum iz teksta evaluacije.
    """
    import re
    import json
    
    
    criterion_pattern = r'<criterion name="([^"]+)">\s*<justification>([^<]+)</justification>\s*<score>([^<]+)</score>\s*</criterion>'
    criteria_matches = re.findall(criterion_pattern, evaluation_text, re.DOTALL)
    
   
    overall_pattern = r'<overall_score>\s*<justification>([^<]+)</justification>\s*<score>([^<]+)</score>\s*</overall_score>'
    overall_match = re.search(overall_pattern, evaluation_text, re.DOTALL)
    
    
    results = {
        "criteria": {},
        "overall": {}
    }
    
   
    for name, justification, score in criteria_matches:
        try:
            score_value = float(score.strip())
        except ValueError:
           
            score_value = score.strip()
            
        results["criteria"][name] = {
            "justification": justification.strip(),
            "score": score_value
        }
    
   
    if overall_match:
        try:
            overall_score = float(overall_match.group(2).strip())
        except ValueError:
            overall_score = overall_match.group(2).strip()
            
        results["overall"] = {
            "justification": overall_match.group(1).strip(),
            "score": overall_score
        }
    
    return json.dumps(results, indent=2, ensure_ascii=False)



def run_full_evaluation(agent_description, agent_system_prompt, original_task, agent_response):
    """
    Izvršava kompletan proces evaluacije agenta.
    """
  
    combined_criteria = generate_criteria(agent_system_prompt)
    
   
    questions_output = generate_questions(agent_system_prompt)
    questions = extract_elements(questions_output, "final_questions")[0] if extract_elements(questions_output, "final_questions") else ""
    
   
    evaluation_output = evaluate_response(agent_description, agent_system_prompt, original_task, agent_response, combined_criteria)
    
   
    evaluation_json = extract_criteria_scores(evaluation_output)
    
    return {
        "combined_criteria": combined_criteria,
        "test_questions": questions,
        "raw_evaluation": evaluation_output,
        "evaluation_json": evaluation_json
    }



if __name__ == "__main__":
   
    AGENT_DESCRIPTION = "AI asistent za pružanje informacija o zdravoj ishrani"
    AGENT_SYSTEM_PROMPT = """
    Ti si stručni nutricionista koji daje savete o zdravoj ishrani.
    1. Uvek navedi naučno dokazane informacije
    2. Izbegavaj promotivne savete za komercijalne dijete
    3. Prilagodi savete individualnim potrebama korisnika
    4. Koristi jednostavan i razumljiv jezik
    5. Ako je pitanje van domena ishrane, ljubazno usmeri korisnika na odgovarajućeg stručnjaka
    """
    ORIGINAL_TASK = "Koje namirnice su dobre za smanjenje upale u zglobovima?"
    AGENT_RESPONSE = """
    Za smanjenje upale u zglobovima, preporučujem sledeće namirnice:
    
    1. Masna riba (losos, sardine, skuša) - bogata omega-3 masnim kiselinama
    2. Kurkuma - sadrži kurkumin koji ima anti-inflamatorna svojstva
    3. Đumbir - ima prirodna anti-inflamatorna dejstva
    4. Bobičasto voće (borovnice, maline) - bogato antioksidansima
    5. Zeleno lisnato povrće - bogato antioksidansima i vitaminima
    6. Orašasti plodovi (naročito orasi) - sadrže omega-3 i antioksidanse
    7. Maslinovo ulje - sadrži oleokantal koji deluje anti-inflamatorno
    
    Ove namirnice je najbolje redovno uključivati u ishranu za dugoročne rezultate. Međutim, ako patite od hroničnih bolova u zglobovima, preporučujem da se obratite lekaru za kompletnu procenu stanja.
    """
    
    
    results = run_full_evaluation(AGENT_DESCRIPTION, AGENT_SYSTEM_PROMPT, ORIGINAL_TASK, AGENT_RESPONSE)
    
   
    print("=== KOMBINOVANI KRITERIJUMI ===")
    print(results["combined_criteria"])
    print("\n=== EVALUACIJA ===")
    print(results["raw_evaluation"])
    print("\n=== JSON REZULTATI ===")
    print(results["evaluation_json"])

print("Evaluator odgovora i glavni proces uspešno postavljeni!")

=== KOMBINOVANI KRITERIJUMI ===

Below are the standard evaluation criteria you must assess in every evaluation:
- Relevance (0–10): Response directly addresses the user's query and is on-topic.
- Accuracy (0–10): Information provided is factually correct and up-to-date.
- Coherence (0–10): Response is logically structured and easy to follow.
- Conciseness (0–10): Answer is appropriately brief without unnecessary elaboration.
- Clarity (0–10): Content is presented in a clear, easily understandable way.
- Completeness (0–10): Response addresses all aspects of the user's query.
- Language Appropriateness (0–10): Language level and tone match the target audience.
- Chain-of-Thought Reasoning (0–10): Response demonstrates logical step-by-step reasoning using expressions like "first", "then", "because", etc.


Additional dynamic criteria specific to this system prompt:

- Scientific Evidence Base (Dynamic): Extent to which the response cites or refers to scientific studies or evidence. [Sca

In [6]:

# Primer 1: Nutricionistički asistent
NUTRITIONIST_DESCRIPTION = "AI asistent za pružanje informacija o zdravoj ishrani"
NUTRITIONIST_PROMPT = """
Ti si stručni nutricionista koji daje savete o zdravoj ishrani.
1. Uvek navedi naučno dokazane informacije
2. Izbegavaj promotivne savete za komercijalne dijete
3. Prilagodi savete individualnim potrebama korisnika
4. Koristi jednostavan i razumljiv jezik
5. Ako je pitanje van domena ishrane, ljubazno usmeri korisnika na odgovarajućeg stručnjaka
"""
NUTRITION_TASK = "Koje namirnice su dobre za smanjenje upale u zglobovima?"
NUTRITION_RESPONSE = """
Za smanjenje upale u zglobovima, preporučujem sledeće namirnice:

1. Masna riba (losos, sardine, skuša) - bogata omega-3 masnim kiselinama
2. Kurkuma - sadrži kurkumin koji ima anti-inflamatorna svojstva
3. Đumbir - ima prirodna anti-inflamatorna dejstva
4. Bobičasto voće (borovnice, maline) - bogato antioksidansima
5. Zeleno lisnato povrće - bogato antioksidansima i vitaminima
6. Orašasti plodovi (naročito orasi) - sadrže omega-3 i antioksidanse
7. Maslinovo ulje - sadrži oleokantal koji deluje anti-inflamatorno

Ove namirnice je najbolje redovno uključivati u ishranu za dugoročne rezultate. Međutim, ako patite od hroničnih bolova u zglobovima, preporučujem da se obratite lekaru za kompletnu procenu stanja.
"""

# Primer 2: Sportski asistent
SPORT_DESCRIPTION = "AI asistent za odgovaranje na pitanja o srpskim NBA igračima"
SPORT_PROMPT = """
Odgovaraj na pitanja vezana za srpske NBA igrače. Ako je pitanje o igračima koji nisu iz Srbije, ljubazno odbij odgovor. Odgovaraj isključivo na srpskom jeziku.
"""
SPORT_TASK = "Ko je prvi srpski igrač koji je igrao u NBA ligi?"
SPORT_RESPONSE = """
Žarko Paspalj je bio prvi srpski košarkaš koji je zaigrao u NBA ligi, nastupajući za San Antonio Spurse u sezoni 1989/1990.
"""

# Primer 3: Zdravstveni asistent
HEALTH_DESCRIPTION = "AI asistent koji daje savete o zdravlju, ishrani i fizičkoj aktivnosti"
HEALTH_PROMPT = """
Odgovaraj na pitanja o zdravlju, ishrani i fizičkoj aktivnosti. Izbegavaj preporuke o lekovima i uvek napomeni da se konsultuje lekar za bilo kakvu terapiju. Odgovori treba da budu stručni, sažeti i napisani jednostavnim jezikom.
"""
HEALTH_TASK = "Koji suplementi mogu pomoći kod anemije?"
HEALTH_RESPONSE = """
Za anemiju se preporučuje suplementacija gvožđem, najčešće u obliku ferosulfata, zajedno sa vitaminom C koji poboljšava apsorpciju. Takođe, suplementi vitamina B12 i folne kiseline mogu biti korisni u određenim vrstama anemije.
"""


def evaluate_and_show_results(agent_description, agent_system_prompt, original_task, agent_response):
    print(f"Evaluacija za: {agent_description}")
    print("-" * 50)
    
   
    print("Generisanje kriterijuma...")
    combined_criteria = generate_criteria(agent_system_prompt)
    
    print("Kriterijumi generisani:")
    print(combined_criteria)
    print("-" * 50)
    
   
    print("Generisanje test pitanja...")
    questions_output = generate_questions(agent_system_prompt)
    questions = extract_elements(questions_output, "final_questions")[0] if extract_elements(questions_output, "final_questions") else ""
    
    print("Test pitanja generisana:")
    print(questions)
    print("-" * 50)
    
    
    print("Evaluacija odgovora...")
    evaluation_output = evaluate_response(agent_description, agent_system_prompt, original_task, agent_response, combined_criteria)
    
   
    evaluation_json = extract_criteria_scores(evaluation_output)
    
    print("Rezultati evaluacije:")
    print(evaluation_json)
    print("-" * 50)
    
    return {
        "combined_criteria": combined_criteria,
        "test_questions": questions,
        "evaluation_output": evaluation_output,
        "evaluation_json": evaluation_json
    }


if __name__ == "__main__":
   
    print("Izaberi primer za evaluaciju:")
    print("1. Nutricionistički asistent")
    print("2. Sportski asistent")
    print("3. Zdravstveni asistent")
    
    choice = input("Unesi broj (1-3): ")
    
    if choice == "1":
        results = evaluate_and_show_results(
            NUTRITIONIST_DESCRIPTION, 
            NUTRITIONIST_PROMPT, 
            NUTRITION_TASK, 
            NUTRITION_RESPONSE
        )
    elif choice == "2":
        results = evaluate_and_show_results(
            SPORT_DESCRIPTION, 
            SPORT_PROMPT, 
            SPORT_TASK, 
            SPORT_RESPONSE
        )
    elif choice == "3":
        results = evaluate_and_show_results(
            HEALTH_DESCRIPTION, 
            HEALTH_PROMPT, 
            HEALTH_TASK, 
            HEALTH_RESPONSE
        )
    else:
        print("Nevažeći izbor. Molimo izaberite broj između 1 i 3.")
    
  

print("Primeri za evaluaciju uspešno postavljeni!")

Izaberi primer za evaluaciju:
1. Nutricionistički asistent
2. Sportski asistent
3. Zdravstveni asistent
Evaluacija za: AI asistent za pružanje informacija o zdravoj ishrani
--------------------------------------------------
Generisanje kriterijuma...
Kriterijumi generisani:

Below are the standard evaluation criteria you must assess in every evaluation:
- Relevance (0–10): Response directly addresses the user's query and is on-topic.
- Accuracy (0–10): Information provided is factually correct and up-to-date.
- Coherence (0–10): Response is logically structured and easy to follow.
- Conciseness (0–10): Answer is appropriately brief without unnecessary elaboration.
- Clarity (0–10): Content is presented in a clear, easily understandable way.
- Completeness (0–10): Response addresses all aspects of the user's query.
- Language Appropriateness (0–10): Language level and tone match the target audience.
- Chain-of-Thought Reasoning (0–10): Response demonstrates logical step-by-step reasonin